# Strands Integration

This notebook demonstrates how to build an agent using the Strands Agents SDK with WRITER’s Palmyra LLM.  


## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)


## Key Concepts (How It Works)

- Use Strands’ `Agent` class to create a self‑contained agent that interacts with a language model and optionally calls tools.
- WRITER’s Palmyra models (e.g., `palmyra-x5`, `palmyra-creative`) can be plugged into Strands via the WriterModel integration.


## Setup
Install the libs:

In [ ]:
%pip install 'strands-agents[writer]'
%pip install strands-agents-tools

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [3]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")


Enter your Writer API key:  ········


## Usage

Once you’ve installed the Strands Agents SDK with WRITER support, you can initialize a Strands agent that uses a Palmyra model from WRITER’s suite.

Here’s how you set it up:

1. Import the `WriterModel` from the Strands integration and specify a model ID (for example, `palmyra‑x5`, `palmyra‑creative`, `palmyra‑med`).
2. Create a Strands `Agent`, passing the model plus any tools you want to enable (such as a calculator or HTTP request tool).
3. Provide a system prompt to define what the agent should do (tone, role, output format).  

> **Best practice:** Use a clear and specific system prompt to define your agent’s role and expected behavior. If you include tools, make it explicit when and how the agent should use them. For many use‑cases a single agent with one model is sufficient and keeps the architecture simple.


In [ ]:
from strands import Agent
from strands.models.writer import WriterModel

writer_model = WriterModel(
    model_id="palmyra-x5"  # or other Palmyra model like palmyra-creative, palmyra-fin, etc.
)

## (Optional) Define Tools

You can optionally pass a list of tools to the agent. For example, if you want the agent to do simple calculations:

In [ ]:
from strands_tools import calculator

tools = [calculator]


This defines the agent’s personality, its reasoning style, and how it should use tools. A good prompt is key to predictable, useful behavior.

In [ ]:
SYSTEM_PROMPT = """
You are a helpful enterprise assistant. Use your knowledge and available tools (if provided)
to answer user queries accurately and helpfully.
"""

agent = Agent(
    model=writer_model,
    tools=tools,         # or [] if no tools
    system_prompt=SYSTEM_PROMPT
)


In [ ]:
response = agent("Explain the benefits of using Strands + WRITER for building AI tools.")
print(response)


# Advanced Multi-Agent Research and Content Creation Workflow

This part provides an **extended example** of a multi-agent workflow 
using the Strands SDK. It goes beyond a basic demonstration to showcase a more 
detailed, real-world application scenario involving multiple specialized agents:

1. **Research Agent**: Leveraging `palmyra-x5` to perform comprehensive web-based research, 
   gather verified information, and summarize insights.

2. **Content Agent**: Utilizing `palmyra-creative` to craft engaging, platform-optimized 
   social media content based on the research findings.

In this extended example, we will cover:

- Importing and configuring necessary modules  
- Initializing and customizing models for each agent  
- Creating agents with specialized roles and workflows  
- Orchestrating a multi-step pipeline for research and content creation  
- Handling intermediate outputs, data formatting, and agent collaboration  
- Scaling and refining the workflow for more complex, real-world use cases  

This notebook serves as both a practical guide and a foundation for building **robust multi-agent systems** 
that combine research, creativity, and automation.

# Import required modules

In [ ]:
from strands import Agent
from strands.models.writer import WriterModel
from strands_tools import http_request

## Initialize the Models

- `palmyra-creative` → for creative social content.
- `palmyra-x5` → for accurate research and factual lookups.


In [ ]:
# Initialize the creative content model
content_model = WriterModel(
    model_id="palmyra-creative",
)

# Initialize the research model
research_model = WriterModel(
    model_id="palmyra-x5",
)


## Define System Prompts

- **Content agent prompt:** instructs the agent to write engaging, platform-specific posts.
- **Research agent prompt:** instructs the agent to gather and summarize factual information from the web.


In [ ]:
# Content agent system prompt
CREATIVE_ASSISTANT_SYSTEM_PROMPT = """
You are the Creative Content Assistant. Your job is to craft imaginative, engaging, and on‑brand content for both X/Twitter and LinkedIn.
Always use vivid language, a friendly tone, and creative flair to delight and inspire the reader.

For each request:
- Write distinct content tailored to each platform's tone and audience
- Label each section clearly as 'LinkedIn' and 'X/Twitter'
- Briefly list the steps or reasoning you used to craft the content
"""

# Research agent system prompt
RESEARCH_ASSISTANT_SYSTEM_PROMPT = """
You are a Researcher Assistant that gathers information from the web. 
1. Determine if the input is a research query or factual claim 
2. Use your research tools (http_request, retrieve) to find relevant information 
3. Include source URLs and keep findings under 500 words
"""


## Create Agents

- **Research Agent** uses `http_request` for live web research.  
- **Content Agent** generates polished social content.


In [ ]:
# Create the research agent
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_SYSTEM_PROMPT,
    callback_handler=None,
    model=research_model,
    tools=[http_request]
)

# Create the content agent
content_agent = Agent(
    system_prompt=CREATIVE_ASSISTANT_SYSTEM_PROMPT,
    callback_handler=None,
    model=content_model
)

## Define the Workflow Function

This function orchestrates both agents:

1. **Research step:** gather factual information.  
2. **Content step:** generate X/Twitter and LinkedIn posts based on the research.


In [ ]:
def run_content_workflow(user_input):
    # Step 1: Research
    researcher_response = research_agent(
        f"Research: '{user_input}'. Use your available tools to gather information from reliable sources.",
    )
    research_findings = str(researcher_response)

    # Step 2: Generate content
    content = content_agent(
        f"Create X/Twitter and LinkedIn social post content based on these findings about '{user_input}': \n\n{research_findings}",
    )

    return content


## Run the Workflow

We will test the workflow using a real announcement from Strands Agents 1.0.


In [ ]:
response = run_content_workflow(
    "Write a creative launch announcement for Strands Agents 1.0 from their announcement here: https://aws.amazon.com/blogs/opensource/introducing-strands-agents-1-0-production-ready-multi-agent-orchestration-made-simple/"
)
print(response)


#  Writer MCP + Strands Agents Cookbook  
## Section: Building a Custom MCP Client for Multi-Step Content Workflows

This recipe shows how to combine:

- **Writer MCP Server** (via `writer-sdk-mcp`)
- **Strands Agents**
- **WriterModel (Palmyra-x5)**
- **MCP Tools over stdio**

to orchestrate multi-step research and content generation workflows.


## Install Dependencies

Install Strands (with Writer support) and the Strands Agent Tools package:

In [ ]:
%pip install 'strands-agents[writer]' strands-agents-tools

## Building a Custom MCP Client with Strands

In [27]:
import os
from strands import Agent
from strands.models.writer import WriterModel
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters



def log_server_init_and_requests(client_factory):
    """Wrap an MCP client to log server start and requests."""
    class LoggingMCPClient:
        def __init__(self):
            self._client = client_factory()

        def __enter__(self):
            print("[LOG] Starting MCP server...")
            self._client.__enter__()
            print("[LOG] MCP server started.")
            return self

        def __exit__(self, exc_type, exc_val, exc_tb):
            self._client.__exit__(exc_type, exc_val, exc_tb)

        def __getattr__(self, name):
            orig_attr = getattr(self._client, name)
            if callable(orig_attr):
                def hooked(*args, **kwargs):
                    print(f"[LOG] MCP request: {name}, args={args}, kwargs={kwargs}")
                    result = orig_attr(*args, **kwargs)
                    print(f"[LOG] MCP Server starting with tools: {',*\n'.join([el.tool_name for el in result])}")
                    return result
                return hooked
            return orig_attr
    return LoggingMCPClient()


# Connect to the Writer MCP server via stdio
writer_mcp_client = log_server_init_and_requests(lambda: MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="npx",
        args=["-y", "writer-sdk-mcp@latest"],
        env={"WRITER_API_KEY": os.environ["WRITER_API_KEY"]}
    )
)))

def content_workflow_client(user_request: str):
    """
    Custom MCP client that orchestrates complex content workflows
    using Writer MCP tools.
    """
    with writer_mcp_client:
        tools = writer_mcp_client.list_tools_sync()
        agent = Agent(
            model=writer_model,
            tools=tools,
            system_prompt="""You are a content workflow assistant that can:
            - Upload and analyze files
            - Generate content using Writer's models
            - Query Knowledge Graphs for research
            - Analyze images and extract insights
            - Chain multiple operations together seamlessly
            Use the available Writer MCP tools to fulfill user requests efficiently."""
        )
        return agent(user_request)


## Example Workflow Request

In [29]:
response = content_workflow_client(
    "Research and create content about sustainable fashion trends, "
    "including a blog post outline and social media content ideas."
)

print("Research & Content:", response)

[LOG] Starting MCP server...
[LOG] MCP server started.
[LOG] MCP request: list_tools_sync, args=(), kwargs={}
[LOG] MCP Server starting with tools: retrieve_applications,*
list_applications,*
generate_content_applications,*
create_applications_jobs,*
retrieve_applications_jobs,*
list_applications_jobs,*
retry_applications_jobs,*
update_applications_graphs,*
list_applications_graphs,*
chat_chat,*
create_completions,*
list_models,*
create_graphs,*
retrieve_graphs,*
update_graphs,*
list_graphs,*
delete_graphs,*
add_file_to_graph_graphs,*
question_graphs,*
remove_file_from_graph_graphs,*
retrieve_files,*
list_files,*
delete_files,*
download_files,*
retry_files,*
upload_files,*
ai_detect_tools,*
context_aware_splitting_tools,*
parse_pdf_tools,*
web_search_tools,*
medical_tools_comprehend,*
translate_translation,*
analyze_vision,*
search_docs
To tackle this task efficiently, I should break it down into smaller steps and use the available Writer tools to fulfill the user's request. My first s

tool execution failed
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/jupyterlab/4.4.7/libexec/lib/python3.13/site-packages/strands/tools/mcp/mcp_client.py", line 512, in call_tool_async
    call_tool_result: MCPCallToolResult = await asyncio.wrap_future(future)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/jupyterlab/4.4.7/libexec/lib/python3.13/site-packages/strands/tools/mcp/mcp_client.py", line 746, in run_async
    return await invoke_event
           ^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/jupyterlab/4.4.7/libexec/lib/python3.13/site-packages/strands/tools/mcp/mcp_client.py", line 506, in _call_tool_async
    return await cast(ClientSession, self._background_thread_session).call_tool(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        name, arguments, read_timeout_seconds
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/opt/homebrew/Cellar/jup

It seems that the application ID provided is not valid. I will need to list available applications and choose a relevant one to create the blog post outline. I will call the `list_applications` function to retrieve a list of available applications.


Tool #3: list_applications
It appears that there are numerous applications available for content generation. For the purpose of creating a blog post outline about sustainable fashion trends, I will choose the "Blog Post Outline" application (though the exact name isn't listed, I'll proceed with a suitable ID).

Let's assume the ID "d0a5bc06-d142-44da-827e-4efa24f18c7c" corresponds to a content generation application that can be used for creating outlines.


Tool #4: generate_content_applications


tool execution failed
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/jupyterlab/4.4.7/libexec/lib/python3.13/site-packages/strands/tools/mcp/mcp_client.py", line 512, in call_tool_async
    call_tool_result: MCPCallToolResult = await asyncio.wrap_future(future)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/jupyterlab/4.4.7/libexec/lib/python3.13/site-packages/strands/tools/mcp/mcp_client.py", line 746, in run_async
    return await invoke_event
           ^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/jupyterlab/4.4.7/libexec/lib/python3.13/site-packages/strands/tools/mcp/mcp_client.py", line 506, in _call_tool_async
    return await cast(ClientSession, self._background_thread_session).call_tool(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        name, arguments, read_timeout_seconds
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/opt/homebrew/Cellar/jup

It seems that the input field name I provided is not valid for the chosen application. Since I don't have more information about the specific application's input fields, I will try a different approach by directly generating content for a blog post outline about sustainable fashion trends using the `create_completions` function.


Tool #5: create_completions
Here's a suggested outline for a blog post on sustainable fashion trends:

**Title:** "Embracing the Future of Fashion: Top Sustainable Trends to Watch"

**I. Introduction**
* Briefly introduce the concept of sustainable fashion and its growing importance
* Mention the environmental and social impact of the fashion industry
* Thesis statement: Sustainable fashion is no longer a niche trend, but a movement that's here to stay.

**II. The Rise of Sustainable Materials**
* Discuss the increasing use of eco-friendly materials in fashion, such as:
	+ Organic cotton
	+ Recycled polyester
	+ Tencel
	+ Piñatex (a non-woven fabric made from